In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
import numpy as np
import nltk
import math
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
stop_words = set(stopwords.words("english"))
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [3]:
import pandas as pd

In [4]:
train_text = pd.read_csv('./data/text_train_deps.csv', encoding='utf8')

In [5]:
train_text.sample()

,Unnamed: 0,description,host_about,house_rules,neighborhood_overview,notes,summary
125045,125025.0,This master bedroom with a private bathroom an...,"Im a 28 year old engineer, national park enthu...",NaN,NaN,NaN,This master bedroom with a private bathroom an...


In [2]:
#combines vocabulary from both entity files
unique_entities = []
for line in open('./data/text_train_deps.csv').readlines():
    word = line.split(',')
    for each in word:
        if each.lower() not in unique_entities:
            unique_entities.append(each.lower())
for line in open('entity_test_deps.csv').readlines():
    word = line.split(',')
    for each in word:
        if each.lower() not in unique_entities:
            unique_entities.append(each.lower())

KeyboardInterrupt: 

In [ ]:
####converting entities to actual strings####
str_unique_entities = []
for each in unique_entities:
    str_unique_entities.append(str(each.strip('"')))

In [102]:
#pulling tokenized sentences from both csv
with open('text_train_deps.csv', 'r') as f:
    entity_test_deps = []
    sents = []
    heading = True
    for line in f:
        if heading == True:
            heading = False
        else:
            sentences = nltk.sent_tokenize(line.decode('ascii', 'ignore'))
            sents.append(sentences)

with open('text_test_deps.csv', 'r') as f:
    entity_test_deps = []
    tok_sents = []
    heading = True
    for line in f:
        if heading == True:
            heading = False
        else:
            sentences = nltk.sent_tokenize(line.decode('ascii', 'ignore'))
            sents.append(sentences)
            tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
            tok_sents.append(tokenized_sentences)

In [439]:
#getting tf and word count
word_count = 0
unique_entity_list = {}
for count in range(len(sents)):
    if len(sents[count]) > 0:
        sentence = sents[count][0].split()
        for word in sentence:
            word_count += 1        
        #counts the frequency of entities in corpus
        for word in str_unique_entities:
            if word in sents[count][0].lower() and word != '':
                if word not in stop_words:
                    if word in unique_entity_list:
                        unique_entity_list[word] += 1
                    else:
                        unique_entity_list[word] = 1                

In [547]:
keys = sorted(unique_entity_list.items(), key=operator.itemgetter(1))
sorted_keys = [key[0] for key in keys]
least_common_1000 = sorted_keys[:1000]
print least_common_1000

['fat buddha', 'microwave coffee', 'vast windows', 'hip potrero hill location', 'busta rhymes', 'hbo one', 'italytown', 'remodel cozy', 'san francisco school', 'mcclaren', 'scottsdale', 'asian market', 'oracleopenworld', 'helsinki', 'bean bag', 'divisidero street', 'bitcoin', 'visitacion valley', 'bowls', 'zynga hq', 'kenny rogers', 'metallica', 'cozy san francisco home', 'kashmir', 'eurecom ecole', 'lego', 'aqua surf shop', 'business insider', 'shared bath available', 'hottub', 'golden gate bridge and golden gate park', 'sparks food truck', 'norcal', 'art history', 'boathouse', 'sunny victorian condo getaway', 'neil diamond', 'starbucks coffee', 'california trolley line', 'downtown civic center', 'dog alert', 'antioch college', 'deyoung art', 'parley', 'zico', 'dave', 'optional twin', 'dogptch', 'new china town', 'late check', 'fairmont hotel', 'top floor view flat', 'slovenia', 'serena williams', 'channel mission bay', 'french bulldog', 'spark social', 'aspen', 'media sales', 'seasid

In [560]:
#loading dependent text
lines = []
filtered_spaces = []
for line in open('text_test_deps.csv').readlines():
    temp_str = line.strip('\r\n')
    #there are empty spaces in text
    if temp_str != "":
        lines.append(temp_str)
        filtered_spaces.append(temp_str)
    else:
    #fills the empty spaces
        lines.append("empty_row")
        
for line in open('text_train_deps.csv').readlines():
    temp_str = line.strip('\r\n')
    #there are empty spaces in text
    if temp_str != "":
        lines.append(temp_str)
        filtered_spaces.append(temp_str)
    else:
    #fills the empty spaces
        lines.append("empty_row")
        

print len(lines)
print len(filtered_spaces)

509898
397754


In [546]:
print lines[5]

- Taylor Swift


In [548]:
#this is only the test set
#this will use nltk's built in sentiment score
#more info on this: http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
#only digested 8 lines to demonstrate. Will continue if needed.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
tfidf_sentiment = []
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

for line in lines:
    word_count = 0
    term_freq = {}
    frequency = []
    senten = nltk.sent_tokenize(line.decode('ascii', 'ignore'))
    token_sent = [nltk.word_tokenize(sentence) for sentence in senten]    
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    
    for tree in chunked:
        frequency.extend(extract_entity_names(tree))
        #gets chunked vocab count
        for each in tree:
            word_count += 1
    
    
    for each in frequency:        
        for word in least_common_5000:
            if word == each.lower():
                if word in term_freq:
                    term_freq[word] += 1
                else:
                    term_freq[word] = 1 
            


    if len(term_freq) > 0:
        for key in term_freq.keys():
            tf = float(term_freq[key])/word_count
            idf = math.log(word_count/(1+float(term_freq[key])))
            tf_idf = tf*idf
        ss = sid.polarity_scores(line)
        tfidf_sentiment.append([[key,tf_idf], ss])
    else:
        ss = sid.polarity_scores(line)
        tfidf_sentiment.append([["no_keys", 0.0], ss])



In [552]:
for each in tfidf_sentiment[:100]:
    print each

[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['dallas', 0.019436619216149344], {'neg': 0.0, 'neu': 0.959, 'pos': 0.041, 'compound': 0.7767}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.6369}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['taylor swift', 0.0], {'neg': 0.0, 'neu': 0.357, 'pos': 0.643, 'compound': 0.2023}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['cole street', 0.01913139252240148], {'neg': 0.048, 'neu': 0.795, 'pos': 0.157, 'compound': 0.9834}]
[['no_keys', 0.0], {'neg': 0.024, 'neu': 

In [563]:
#this is both train and test dependency together
#this will use nltk's built in sentiment score
#more info on this: http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html
#only digested 8 lines to demonstrate. Will continue if needed.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
tfidf_sentiment = []
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

for line in lines:
    word_count = 0
    term_freq = {}
    frequency = []
    senten = nltk.sent_tokenize(line.decode('ascii', 'ignore'))
    token_sent = [nltk.word_tokenize(sentence) for sentence in senten]    
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    
    for tree in chunked:
        frequency.extend(extract_entity_names(tree))
        #gets chunked vocab count
        for each in tree:
            word_count += 1
    
    
    for each in frequency:        
        for word in least_common_5000:
            if word == each.lower():
                if word in term_freq:
                    term_freq[word] += 1
                else:
                    term_freq[word] = 1 
            


    if len(term_freq) > 0:
        for key in term_freq.keys():
            tf = float(term_freq[key])/word_count
            idf = math.log(word_count/(1+float(term_freq[key])))
            tf_idf = tf*idf
        ss = sid.polarity_scores(line)
        tfidf_sentiment.append([[key,tf_idf], ss])
    else:
        ss = sid.polarity_scores(line)
        tfidf_sentiment.append([["no_keys", 0.0], ss])

In [566]:
for each in tfidf_sentiment[509800:509898]:
    print each

[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['thx', 0.016525922765941697], {'neg': 0.031, 'neu': 0.863, 'pos': 0.106, 'compound': 0.9508}]
[['no_keys', 0.0], {'neg': 0.041, 'neu': 0.83, 'pos': 0.129, 'compound': 0.9808}]
[['no_keys', 0.0], {'neg': 0.013, 'neu': 0.909, 'pos': 0.077, 'compound': 0.8885}]
[['no_keys', 0.0], {'neg': 0.056, 'neu': 0.808, 'pos': 0.136, 'compound': 0.9826}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 0.914, 'pos': 0.086, 'compound': 0.9741}]
[['no_keys', 0.0], {'neg': 0.007, 'neu': 0.867, 'pos': 0.126, 'compound': 0.9846}]
[['no_keys', 0.0], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[['sf giants', 0.009845604974325923], {'neg': 0.042, 'neu': 0.786, 'pos': 0.172, 'compound': 0.9969}]
[['no_keys', 0.0], {'neg': 0.027, 'neu': 0.808, 'pos': 0.165, 'compound': 0.9967}]
[['no_keys', 0.0], {'neg': 0.022, 'neu': 0.852, 'pos': 0.126, 'compound': 0.9682}]
[['no_keys', 0.0], {'neg': 0.009, 'neu': 0.766, 'pos': 0.225, 'compound': 0.

In [567]:
import csv

with open('tfidf_sentiment.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(tfidf_sentiment)